In [3]:
from typing import List
import requests
import json
import pandas as pd
import numpy as np
import time
import sys
from collections.abc import MutableMapping,Iterable
import yaml
import datetime


def get_API_response(url):
    response_API = requests.get(url)
    #print('Status Code-', response_API.status_code)
    #print('Loading Data--')
    start = time.time()
    raw_data = response_API.text
    # Parse Json
    data = json.loads(raw_data)
    print('Time taken to load data-',round((time.time()-start),3))
    return data
# Load api-keys
config_stream = open("../../config.yml",'r')
config = yaml.load(config_stream, Loader=yaml.BaseLoader)

etherscan_API_key = config['api_key']['etherscan']
etherscan_APIPro_key = config['api_key']['etherscan_pro']

R_ExtractDate = datetime.datetime.today().strftime("%Y-%m-%d")
frax_start_date= '2020-12-21'
frax_contract_address = "0x853d955aCEf822Db058eb8505911ED77F175b99e"

# Stats

## Get Historical Ether Price

In [6]:
# Get Historical Ether Price
url_eth_hist_mcap_string ="https://api.etherscan.io/api?module=stats&action=ethdailymarketcap&startdate={0}&enddate={1}&sort={2}&apikey={3}" 
start_date = frax_start_date
end_date = R_ExtractDate # current date
order = 'asc'
url_eth_hist_mcap = url_eth_hist_mcap_string.format(start_date, end_date, order, etherscan_APIPro_key)
raw_eth_hist_price_mcap_supply = get_API_response(url_eth_hist_mcap)
pdf_eth_hist_price_mcap_supply = pd.DataFrame(raw_eth_hist_price_mcap_supply['result'])

Status Code- 200
Loading Data--
Time taken to load data- 0.008


In [10]:
pdf_eth_hist_price_mcap_supply.tail()

,UTCDate,unixTimeStamp,supply,marketCap,price
592,2022-08-05,1659657600,121846161.281250,211619.975990049375,1736.78
593,2022-08-06,1659744000,121859518.593750,206024.2323008953125,1690.67
594,2022-08-07,1659830400,121872992.093750,207249.897975105625,1700.54
595,2022-08-08,1659916800,121886409.281250,216712.8168379696875,1777.99
596,2022-08-09,1660003200,121899717.843750,207645.1983722221875,1703.41


# Tokens

## Get Token Holder List by Contract Address

In [39]:

# Get Historical Ether Price
url_tokenHolders_forContract ="https://api.etherscan.io/api?module=token&action=tokenholderlist&contractaddress={0}&page={1}&offset={2}&apikey={3}"  
contract_address = "0x853d955aCEf822Db058eb8505911ED77F175b99e"#frax_contract_address# "0x0e09fabb73bd3ade0a17ecc321fd13a19e81ce82"
page = 1
offset = 100000
url_tokenHolders_forContract = url_tokenHolders_forContract.format(frax_contract_address, page, offset, etherscan_APIPro_key)
raw_tokenHolders_forContract = get_API_response(url_tokenHolders_forContract)
pdf_tokenHolders_forContract = pd.DataFrame(raw_tokenHolders_forContract['result'])

# Number of Frax holders = 6628
# pdf_tokenHolders_forContract.shape (6628, 2)

Status Code- 200
Loading Data--
Time taken to load data- 0.005


In [40]:
pdf_tokenHolders_forContract.shape

(6623, 2)

In [21]:
pdf_tokenHolders_forContract.head()

,TokenHolderAddress,TokenHolderQuantity
0,0x642a101709e48d986844656f6ec7586bed71bedd,1134618103536574
1,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,112483374916467338742180
2,0xfd0a40bc83c5fae4203dec7e5929b446b07d1c76,25427841096616591795369
3,0xe1573b9d29e2183b1af0e743dc2754979a40d237,34155278528519362062218799
4,0x69863cdd8a25bae9a57271f7a29a76764da87f1a,4014225114863263981196


In [38]:
pdf_tokenHolders_forContract[pdf_tokenHolders_forContract['TokenHolderAddress'].str.contains("000000000000000")]

,TokenHolderAddress,TokenHolderQuantity
5,0x0000000000000000000000000000000000000001,166180961677277
4068,0x000000000000000000000000000000000000dead,88951933502969935


In [41]:
str(riju_address).lower()

'0x65cad075628f4424120dc76e1d683e925446b8cc'

In [43]:
amir_address = "0x6e74053a3798e0fC9a9775F7995316b27f21c4D2" # no result
riju_address = "0x65cAD075628F4424120Dc76E1D683E925446B8CC" # no result
pdf_tokenHolders_forContract[pdf_tokenHolders_forContract['TokenHolderAddress']==str(amir_address).lower()]#"0x65cAD075628F4424120Dc76E1D683E925446B8CC"]

,TokenHolderAddress,TokenHolderQuantity
5085,0x6e74053a3798e0fc9a9775f7995316b27f21c4d2,4950000000000000000000


## Get Historical ERC20-Token Account Balance for TokenContractAddress by BlockNo

In [30]:
# get historical token for account
url_hist_token_account="https://api.etherscan.io/api?module=account&action=tokenbalancehistory&contractaddress={0}&address={1}&apikey={2}"  #&blockno={2}
contract_address = frax_contract_address# 0x57d90b64a1a57749b0f932f1a3395792e12e7055
address = "0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d" #0xe04f27eb70e025b78871a2ad7eabe85e61212761
# blockno = 
url_hist_token_account = url_hist_token_account.format(frax_contract_address, address,  etherscan_APIPro_key) # blockno,
raw_hist_token_account = get_API_response(url_hist_token_account)
# pdf_hist_token_account = pd.DataFrame(raw_hist_token_account['result'])

# Number of Frax holders = 6628
# pdf_tokenHolders_forContract.shape (6628, 2)

Status Code- 200
Loading Data--
Time taken to load data- 0.0


In [31]:
raw_hist_token_account

{'status': '0',
 'message': 'NOTOK',
 'result': 'Error! Invalid BlockNo. The selected blockno should be > 0 and < 15315095'}

# Blocks

## Get nearest block to timestamp; Frax starting block - 11493365


In [27]:
# Approx frax start timestamp
timestamp = 1607558400 #1608508800
url = (f"""https://api.etherscan.io/api
   ?module=block
   &action=getblocknobytime
   &timestamp={timestamp}
   &closest=before
   &apikey={etherscan_APIPro_key}""")
url = url.replace('\n   ','')
raw_data = get_API_response(url)

# raw_data['result'] #'11421842'

Status Code- 200
Loading Data--
Time taken to load data- 0.0


# Accounts

## Get a list of 'ERC20 - Token Transfer Events' by Address

In [12]:
startblock = 11720000 #11493365 #11491842 #11493366
# Found this - {11493365} the starting block for frax for all zero address 
endblock = 15340347 #startblock + 50000
# address = "0x0000000000000000000000000000000000000000"
# address = "0x57f627f2e848f05ffc0d10c2ab45870922b3b429"
address = "0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d"
url = (f"""https://api.etherscan.io/api
        ?module=account
        &action=tokentx
        &contractaddress={frax_contract_address}
        &address={address}
        &page=1
        &offset=-1
        &startblock={startblock}
        &endblock={endblock}
        &sort=asc
        &apikey={etherscan_APIPro_key}
        """)
url = url.replace('\n        ','')
raw_data = get_API_response(url)
pdf_raw = pd.DataFrame(raw_data['result'])
# pdf_raw.columns
# ['blockNumber', 'timeStamp', 'hash', 'nonce', 'blockHash', 'from',
#        'contractAddress', 'to', 'value', 'tokenName', 'tokenSymbol',
#        'tokenDecimal', 'transactionIndex', 'gas', 'gasPrice', 'gasUsed',
#        'cumulativeGasUsed', 'input', 'confirmations']
cols_selected = ['blockNumber', 'timeStamp','from','to', 'value', \
    'transactionIndex', 'gas', 'gasPrice', 'gasUsed', 'cumulativeGasUsed']
pdf_raw_2 = pdf_raw[cols_selected]
    

Status Code- 200
Loading Data--
Time taken to load data- 0.079


In [13]:
pdf_raw_2

,blockNumber,timeStamp,from,to,value,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed
0,11720049,1611514045,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,0x860bd2dba9cd475a61e6d1b45e16c365f6d78f66,14733567368267813390253,94,727180,103000000000,432364,9375400
1,11720064,1611514352,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,0x351307bf5b6b3feccf0f325536626e90093a7bc2,12304517670724767129288,135,301758,86000001459,244649,11303111
2,11720067,1611514390,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,0xe069cb01d06ba617bcdf789bf2ff0d5e5ca20c71,504228402222354875185,118,234814,98000000000,180786,9053614
3,11720074,1611514566,0x0d067279148a4003711d662d79a8d5c9117920b2,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,13946970896165863668093,169,409556,92000000000,341180,11961197
4,11720152,1611515466,0x73af3bd4228fb4034c8c3d63791dc51b2de28363,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,14294285744281402654150,37,834746,124000100000,544369,2260367
...,...,...,...,...,...,...,...,...,...,...
9995,13075572,1629641284,0xe1573b9d29e2183b1af0e743dc2754979a40d237,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,16440813898111914056711,1,410700,63000000000,330945,514450
9996,13075573,1629641306,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,0xc63b0708e2f7e69cb8a1df0e1389a98c35a76d52,18497680229311123239002,0,267123,145461466045,154598,154598
9997,13076815,1629657778,0xe1573b9d29e2183b1af0e743dc2754979a40d237,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,7681901391429375087522,86,300216,40446744480,237101,7386738
9998,13076859,1629658337,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,0xfd0a40bc83c5fae4203dec7e5929b446b07d1c76,8297368913723628089609,2,435612,46386740414,346028,661345


In [60]:
pdf_raw_2.head(100)

,blockNumber,timeStamp,from,to,value,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed
0,11493365,1608508752,0x0000000000000000000000000000000000000000,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,1998970008000,10,2000000,50000000000,300007,1581136
1,11493366,1608508760,0x0000000000000000000000000000000000000000,0xf97b96d0749001d65c09e21a86a2ac156be704d0,449768251800000000000,4,501112,602905595865,217194,412075
2,11493366,1608508760,0x0000000000000000000000000000000000000000,0x934adbb72e3a8cbbd4dc0934fd2bd579948eaefb,14992275060000000000000,8,160881,250000000000,106795,1059762
3,11493366,1608508760,0x0000000000000000000000000000000000000000,0x4d928359e8556197f161f4927a928dfde9c3d1a8,9994850040000000000000,18,160881,160000000000,106795,2008989
4,11493366,1608508760,0x0000000000000000000000000000000000000000,0x99d34444d97031292955c1d686802909750c8707,19989700080000000000000,21,160863,150000000000,106783,2173760
...,...,...,...,...,...,...,...,...,...,...
95,11493370,1608508858,0x0000000000000000000000000000000000000000,0x1d82c486cc5f243f379f52b5ea7a205d091dc7c5,1599307471661031132000,6,160863,47970000256,91783,832282
96,11493370,1608508858,0x0000000000000000000000000000000000000000,0x067ef3c16f47cca6a3acb5942c6d4bc89ed0a8b1,5179513783696425348000,80,160899,44000000000,91807,10437540
97,11493371,1608508863,0x0000000000000000000000000000000000000000,0xcb114805b901f7a9c38d5675272ef26459a7d805,2998455012000000000000,3,160827,200000000000,106759,444953
98,11493371,1608508863,0x0000000000000000000000000000000000000000,0x9b3f49a186670194f625199b822fcbdfd3feacf7,49974250200000000000000,10,160881,120000000000,106795,1001764


In [65]:
pdf_raw_2

,blockNumber,timeStamp,from,to,value,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed
0,11493365,1608508752,0x0000000000000000000000000000000000000000,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,1998970008000,10,2000000,50000000000,300007,1581136
1,11493365,1608508752,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,1998970008000,10,2000000,50000000000,300007,1581136
2,11493512,1608510718,0x0000000000000000000000000000000000000000,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,1998970008000000000000,190,2000000,73000000000,284748,8464297
3,11493512,1608510718,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,1998970008000000000000,190,2000000,73000000000,284748,8464297
4,11493559,1608511382,0x0000000000000000000000000000000000000000,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,9000008722287899316000,188,2000000,75000000000,273662,9192505
5,11493559,1608511382,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,9000008722287899316000,188,2000000,75000000000,273662,9192505


In [87]:
pdf_raw_2

,blockNumber,timeStamp,from,to,value,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed
0,11493365,1608508752,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,1998970008000,10,2000000,50000000000,300007,1581136
1,11493366,1608508760,0xc930467ee7e682eca73383c1ffafeb02c85a3e15,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,793594608364759068000,29,9000000,120100000000,400144,2898133
2,11493366,1608508760,0xc930467ee7e682eca73383c1ffafeb02c85a3e15,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,49802730976590482770104,29,9000000,120100000000,400144,2898133
3,11493366,1608508760,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,0x78a55b9b3bbeffb36a43d9905f654d2769dc55e8,454433179515276043857,31,721700,120100000000,317188,3267679
4,11493366,1608508760,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,0x860bd2dba9cd475a61e6d1b45e16c365f6d78f66,665752155135155314146,34,612908,120100000000,348092,3755230
...,...,...,...,...,...,...,...,...,...,...
1045,11494651,1608526607,0xe5a8654631b3729f73ca3503ba09a5d3e11b46da,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,24831492201159511652629,86,165418,60000000000,128987,5169159
1046,11494653,1608526655,0x178ca4511cb15bedf58eed96cdea2bead2f414e6,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,10949892890912220941732,99,137736,45000000000,103942,6176544
1047,11494654,1608526661,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,0xf2df969f59b2c86e4b230da88918cdebcfc4ccbc,49311468692709263904623,22,141017,62000000000,133990,1471266
1048,11494658,1608526712,0xfd0a40bc83c5fae4203dec7e5929b446b07d1c76,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,2068998487182252379728,81,828412,50100000000,426277,5977188


## Automate the process

In [4]:
def get_hist_frax_transactions(address: str, startblock: int, endblock: int) -> pd.DataFrame:
    try: 
        url = (f"""https://api.etherscan.io/api
                ?module=account
                &action=tokentx
                &contractaddress={frax_contract_address}
                &address={address}
                &page=1
                &offset=-1
                &startblock={startblock}
                &endblock={endblock}
                &sort=asc
                &apikey={etherscan_APIPro_key}
                """)
        url = url.replace('\n                ','')
        raw_data = get_API_response(url)
        pdf_raw = pd.DataFrame(raw_data['result'])
        cols_selected = ['blockNumber', 'timeStamp','from','to', 'value', \
            'transactionIndex', 'gas', 'gasPrice', 'gasUsed', 'cumulativeGasUsed']
        pdf_raw_2 = pdf_raw[cols_selected]
        return pdf_raw_2
    except Exception as e:
        print(type(e).__name__, e.args)
            

In [4]:
# startblock = 11493365 #11720000 #11493365 #11491842 #11493366
# # Found this - {11493365} the starting block for frax for all zero address 
# endblock = 15340347 #startblock + 50000
# # address = "0x0000000000000000000000000000000000000000"
# address = "0x57f627f2e848f05ffc0d10c2ab45870922b3b429"
# #address = "0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d"
# pdf_op = get_hist_frax_transactions(address, startblock, endblock)

In [5]:
# Start with starting address
all_address_set = ['0x0000000000000000000000000000000000000000']
# a list that hold address that have been visited
visited_address_set = []
list_pdf_op = []
count = 0
# for each address get the whole data
while ((len(all_address_set) > 0) and count < 50000): #is not empty:
    count +=1
    print("count-------------------------- ",count)
    # Get the address- 
    address = all_address_set.pop()
    # if the address is not alredy visited then only get data for it
    if address not in visited_address_set:
        # The starting block for frax for all zero address  - 11493365
        startblock = 11493365
        # block at current timestamp
        endblock = 15345872 #15340347
        pdf_row_count = 10000 # implementation depth
        list_pdf_op_address = []
        # get the output if the output size is 10000;
        # run this many times by changing starting block to get the whole data for an address
        while pdf_row_count == 10000: # This means the output is getting limited
            print(f'getting data for address- {address}, start_blk -{startblock}, end_blk -{endblock}')
            pdf_op = get_hist_frax_transactions(address, startblock, endblock)
            pdf_row_count = pdf_op.shape[0]
            print('number of rows -',pdf_row_count)
            startblock =  pdf_op['blockNumber'].max()
            list_pdf_op_address.append(pdf_op)
        # output data for each address
        pdf_op_address = pd.concat(list_pdf_op_address)
        # output list contating all addresses
        list_pdf_op.append(pdf_op_address)
        # update visited address lists - 
        visited_address_set.append(address)
        
        new_address_from = set(pdf_op_address['from'].values)
        # print(f'new from-addresses - {len(new_address_from)}')
        new_address_to = set(pdf_op_address['to'].values)
        # print(f'new to-addresses - {len(new_address_to)}')
        new_address_set = new_address_from.union(new_address_to)
        print(f'new unique-addresses - {len(new_address_set)}')
        # Union with existing address set as the new ones can be a repeat 
        all_address_set = list(set(set(all_address_set).union(new_address_set)) - set(visited_address_set)) 

    else:
        print(f'address {address} already visited')
    if count % 100 == 0:
        print('saving data at count - ',count)
        pdf_op_result = pd.concat(list_pdf_op)
        pdf_op_result.drop_duplicates(inplace=True)
        pdf_op_result.to_csv("../data_checks/frax_hist_trans_etherscanner/data_"+str(count)+".csv", index=False)
        list_pdf_op = [] # make the list empty again to avoid memeory issue
        pdf_visited_address_set = pd.DataFrame(visited_address_set, columns=['visited_address'])    
        pdf_visited_address_set.to_csv("../data_checks/frax_hist_trans_etherscanner/visisted_address_"+str(count)+".csv", index=False)
        pdf_address_queue = pd.DataFrame(all_address_set, columns=['address'])    
        pdf_address_queue.to_csv("../data_checks/frax_hist_trans_etherscanner/address_queue_"+str(count)+".csv", index=False)
        
        

if len(list_pdf_op)>0:
    pdf_op_result = pd.concat(list_pdf_op)
    pdf_op_result.drop_duplicates(inplace=True)
    pdf_op_result.to_csv("../data_checks/frax_hist_trans_etherscanner/data_final.csv", index=False)
    pdf_visited_address_set = pd.DataFrame(visited_address_set, columns=['visited_address'])    
    pdf_visited_address_set.to_csv("../data_checks/frax_hist_trans_etherscanner/visisted_address_final.csv", index=False)
    pdf_address_queue = pd.DataFrame(all_address_set, columns=['address'])    
    pdf_address_queue.to_csv("../data_checks/frax_hist_trans_etherscanner/address_queue_final.csv", index=False)
        





count--------------------------  1
getting data for address- 0x0000000000000000000000000000000000000000, start_blk -11493365, end_blk -15345872
Time taken to load data- 0.015
number of rows - 3810
new unique-addresses - 881
count--------------------------  2
getting data for address- 0xd2050719ea37325bdb6c18a85f6c442221811fac, start_blk -11493365, end_blk -15345872
Time taken to load data- 0.0
number of rows - 8
new unique-addresses - 7
count--------------------------  3
getting data for address- 0x1bef2b5d795e1d95ad324c6ff12053ac693097d2, start_blk -11493365, end_blk -15345872
Time taken to load data- 0.0
number of rows - 17
new unique-addresses - 8
count--------------------------  4
address 0xd2050719ea37325bdb6c18a85f6c442221811fac already visited
count--------------------------  5
getting data for address- 0xc91bb4b0696e3b48c0c501b4ce8e7244fc363a79, start_blk -11493365, end_blk -15345872
Time taken to load data- 0.0
number of rows - 8
new unique-addresses - 6
count-----------------

AttributeError: 'NoneType' object has no attribute 'shape'